In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .enableHiveSupport() \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/17 06:31:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.format('jdbc') \
    .option('url', 'jdbc:postgresql://metastore:5432/metastore') \
    .option('dbtable', 'information_schema.tables') \
    .option('user', 'jupyter') \
    .option('password', 'jupyter') \
    .load()

In [5]:
df.show(5)

+-------------+------------+------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|  table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|    metastore|      public|         DBS|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|    metastore|      public|    DB_PRIVS|BASE TABLE|                        null|                null|                     n

In [6]:
df.count()

246

In [7]:
df.schema

StructType([StructField('table_catalog', StringType(), True), StructField('table_schema', StringType(), True), StructField('table_name', StringType(), True), StructField('table_type', StringType(), True), StructField('self_referencing_column_name', StringType(), True), StructField('reference_generation', StringType(), True), StructField('user_defined_type_catalog', StringType(), True), StructField('user_defined_type_schema', StringType(), True), StructField('user_defined_type_name', StringType(), True), StructField('is_insertable_into', StringType(), True), StructField('is_typed', StringType(), True), StructField('commit_action', StringType(), True)])

In [8]:
!hive -e 'DROP TABLE IF EXISTS information_schema_tables'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = f074c6c5-cadd-405b-93a0-c30f088eb449

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 78901d3f-c3e8-4f55-84c0-d623d602e598
OK
Time taken: 0.745 seconds


In [9]:
!hive -e 'CREATE TABLE information_schema_tables \
    ( \
    table_catalog STRING, \
    table_schema STRING, \
    table_name STRING, \
    table_type STRING, \
    self_referencing_column_name STRING, \
    reference_generation STRING, \
    user_defined_type_catalog STRING, \
    user_defined_type_schema STRING, \
    user_defined_type_name STRING, \
    is_insertable_into STRING, \
    is_typed STRING, \
    commit_action STRING \
    ) \
    STORED AS ORC'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 83dc793b-8686-497a-a4ab-433eeb05aedb

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 091f5eee-2678-4559-b1ff-4117618fd29e
OK
Time taken: 0.802 seconds


In [10]:
df.createOrReplaceTempView('df')

In [11]:
spark.sql('INSERT OVERWRITE TABLE information_schema_tables SELECT * FROM df')

DataFrame[]

In [12]:
spark.table('information_schema_tables').show(5)

+-------------+------------+------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|  table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|    metastore|      public|         DBS|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|    metastore|      public|    DB_PRIVS|BASE TABLE|                        null|                null|                     n

In [13]:
spark.table('information_schema_tables').count()

246